# Practical Exam: Supermarket Loyalty

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [99]:
# Use this cell to write your code for Task 1
import pandas as pd

In [100]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    ## TASK 1
    
    # replace 'joininig_month' missing values with 'Unknown'
    df['joining_month'].fillna('Unknown',inplace=True)
    # cnverting dtypes 'first_month' to float 
    df['first_month'] = pd.to_numeric(df['first_month'],errors='coerce')
    # replacing missing value with 0 in 'first_month' column
    df['first_month'].fillna(0,inplace=True)
    # continuous columns rounding upto 2 decimal points
    df[['spend','first_month']]= df[['spend','first_month']].round(2)
    # string in promotion column converting to same case ('title')
    df['promotion'] = df['promotion'].str.title()
    
    return df

In [101]:
clean_data = wrangle('loyalty.csv')

In [102]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customer_id           1246 non-null   int64  
 1   spend                 1246 non-null   float64
 2   first_month           1246 non-null   float64
 3   items_in_first_month  1246 non-null   int64  
 4   region                1246 non-null   object 
 5   loyalty_years         1246 non-null   object 
 6   joining_month         1246 non-null   object 
 7   promotion             1246 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 78.0+ KB


# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [103]:
# Use this cell to write your code for Task 2

In [104]:
df = clean_data.copy()

In [105]:
spend_by_years = df.groupby('loyalty_years')['spend'].aggregate(['mean','var']).reset_index().round(2)
spend_by_years.rename(columns={'mean':'avg_spend',
                              'var':'var_spend'},inplace=True)
spend_by_years

,loyalty_years,avg_spend,var_spend
0,0-1,110.56,9.30
1,1-3,129.31,9.65
2,10+,117.41,16.72
3,3-5,124.55,11.09
4,5-10,135.15,14.10


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [106]:
# Use this cell to write your code for Task 3

In [107]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [108]:
train_data = wrangle("train.csv")
test_data = pd.read_csv("test.csv")

train_dum = pd.get_dummies(train_data,columns=['region','loyalty_years','joining_month','promotion'])
test_dum = pd.get_dummies(test_data,columns=['region','loyalty_years','joining_month','promotion'])

X_train = train_dum.drop(columns='spend')
y_train = train_dum['spend']

base_model = LinearRegression()
base_model.fit(X_train,y_train)
base_pred = base_model.predict(test_dum)

test_data['spend'] = base_pred.round(2)
base_result = test_data[['customer_id','spend']]
base_result.head()

,customer_id,spend
0,5,140.71
1,7,148.74
2,16,140.82
3,17,150.66
4,19,153.64


# Task 4

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [109]:
# Use this cell to write your code for Task 4

In [110]:
comp_model = DecisionTreeRegressor()
comp_model.fit(X_train,y_train)
comp_pred = comp_model.predict(test_dum)

test_data['spend'] = comp_pred.round(2)
compare_result = test_data[['customer_id','spend']]
compare_result.head()

,customer_id,spend
0,5,140.20
1,7,148.35
2,16,141.02
3,17,151.16
4,19,153.68
